In [1]:
# Relevant module imports and installs
%pip install pulp pandas brotli fuzzywuzzy
import pandas as pd
import pulp as plp
import sys 
import os
from collections import defaultdict
from fuzzywuzzy import process
import json


[notice] A new release of pip is available: 23.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Get the absolute path to the directory containing the Python file
module_path = os.path.abspath(os.path.join('..', '..'))

# Add the directory to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

# Import the data retrieval functions
from challenge_data import get_bootstrap_static_data, get_gameweek_data, update_with_gameweek_cost

# Get the actual GW12 data
bootstrap_data = get_bootstrap_static_data()
gw12_data = update_with_gameweek_cost(get_gameweek_data(12, bootstrap_data), '2024-25', 12)

In [3]:
# Create optimization model
model = plp.LpProblem("GW12_Hindsight_Optimization", plp.LpMaximize)

# Get number of players
player_count = len(gw12_data)

# Create binary variables for lineup and captain selection
lineup = [plp.LpVariable(f"lineup_{i}", 0, 1, plp.LpBinary) for i in range(player_count)]
captain = [plp.LpVariable(f"captain_{i}", 0, 1, plp.LpBinary) for i in range(player_count)]

# Objective function: Maximize total points (double for captain)
model += plp.lpSum([lineup[i] * gw12_data.loc[i, 'Points'] + captain[i] * gw12_data.loc[i, 'Points'] for i in range(player_count)])

# Constraints
# Exactly 7 players
model += plp.lpSum(lineup) == 7

# Exactly one captain
model += plp.lpSum(captain) == 1

# Captain must be in the lineup
for i in range(player_count):
    model += captain[i] <= lineup[i]

# Exactly 1 Goalkeeper
model += plp.lpSum([lineup[i] for i in range(player_count) if gw12_data.loc[i, 'Position'] == 'Goalkeeper']) == 1

# Between 2-3 Defenders
model += plp.lpSum([lineup[i] for i in range(player_count) if gw12_data.loc[i, 'Position'] == 'Defender']) >= 2
model += plp.lpSum([lineup[i] for i in range(player_count) if gw12_data.loc[i, 'Position'] == 'Defender']) <= 3

# Between 1-3 Midfielders
model += plp.lpSum([lineup[i] for i in range(player_count) if gw12_data.loc[i, 'Position'] == 'Midfielder']) >= 1
model += plp.lpSum([lineup[i] for i in range(player_count) if gw12_data.loc[i, 'Position'] == 'Midfielder']) <= 3

# Between 1-3 Forwards
model += plp.lpSum([lineup[i] for i in range(player_count) if gw12_data.loc[i, 'Position'] == 'Forward']) >= 1
model += plp.lpSum([lineup[i] for i in range(player_count) if gw12_data.loc[i, 'Position'] == 'Forward']) <= 3

# Budget constraint of £45m
model += plp.lpSum([lineup[i] * gw12_data.loc[i, 'Cost'] for i in range(player_count)]) <= 45

# Solve the problem
model.solve(plp.PULP_CBC_CMD(msg=False))

# Function to print players by position
def print_players_by_position(players_dict):
    total_points = 0
    total_cost = 0
    for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
        if position in players_dict:
            print(f"\n{position}:")
            for player in players_dict[position]:
                captain_str = " (C)" if player['Captain'] else ""
                points = player['Points'] * (2 if player['Captain'] else 1)
                print(f"  {player['Name']}{captain_str} - {player['Team']} - Cost: {player['Cost']}m - Points: {points}")
                total_points += points
                total_cost += player['Cost']
    print(f"\nTotal Points: {round(total_points, 2)}")
    print(f"Total Cost: {round(total_cost, 2)}m")

# Print the results
print("Status:", plp.LpStatus[model.status])

print("\nOptimal Lineup:")
selected_players = defaultdict(list)
for i in range(player_count):
    if lineup[i].value() == 1:
        player = gw12_data.loc[i]
        selected_players[player['Position']].append({
            'Name': player['Name'],
            'Team': player['Team'],
            'Cost': player['Cost'],
            'Points': player['Points'],
            'Captain': captain[i].value() == 1
        })

print_players_by_position(selected_players)


Status: Optimal

Optimal Lineup:

Goalkeeper:
  Flekken - Brentford - Cost: 4.5m - Points: 9

Defender:
  Wan-Bissaka - West Ham - Cost: 4.4m - Points: 15
  Pedro Porro - Spurs - Cost: 5.5m - Points: 14

Midfielder:
  Saka - Arsenal - Cost: 10.1m - Points: 13
  Maddison (C) - Spurs - Cost: 7.5m - Points: 32
  I.Sarr - Crystal Palace - Cost: 5.7m - Points: 13

Forward:
  Cunha - Wolves - Cost: 6.8m - Points: 16

Total Points: 112
Total Cost: 44.5m
